In [ ]:
pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 38.8 MB/s eta 0:00:00


In [ ]:
x = [i for i in range(0,20)]
k = 5
c =[5,10,7,8,6,9,12,15,20,17,3,11,14,18,4,16,19,13,2,1]
r =[10,15,9,14,11,13,20,25,30,28,8,17,22,26,12,24,27,19,7,5]
capacidade = 16
raiz_capacidade = 16**(1/2)
b = 50

In [ ]:
def fob(var):
  value = 0
  for i in range(0,20):
    value += r[i]*var[i]
  return -value

def restricao_1(var):
  value = 0
  for i in range(0,20):
    value += var[i]
  return value

def restricao_2(var):
  value = 0
  for i in range(0,20):
    value += var[i]
  return value

def restricao_3(var):
  value = 0
  for i in range(0,20):
    value += var[i]*c[i]
  return value



In [ ]:
###https://coin-or.github.io/pulp/main/includeme.html


import pulp

prob = pulp.LpProblem("Análise de Investimento", pulp.LpMinimize)

for i in range(0,20):
  x[i] = pulp.LpVariable(f'x{i+1}', 0, 1, cat='Integer')



prob += fob(x)
prob += restricao_1(x)<=k
prob += restricao_2(x)<=raiz_capacidade
prob += restricao_3(x)<=b
prob.solve()
print("Status:", pulp.LpStatus[prob.status])
valor_otimo = -1*pulp.value(prob.objective)

print(f'Valor Ótimo: {valor_otimo}\n')
result_var = []
for i in range(0,20):
  result_var.append(pulp.value(x[i]))
cont = 0
print(result_var)
for i,j in enumerate(result_var):
  status = status = 'Será Utilizado' if j == 1 else "Não será Utilizado"
  print(f'O Investimento {i+1} {status}\n')
  if j == 1:
    cont += 1
print(f'Serão utilizados {cont} investimentos\n')

Status: Optimal
Valor Ótimo: 87.0

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
O Investimento 1 Não será Utilizado

O Investimento 2 Não será Utilizado

O Investimento 3 Não será Utilizado

O Investimento 4 Não será Utilizado

O Investimento 5 Não será Utilizado

O Investimento 6 Não será Utilizado

O Investimento 7 Não será Utilizado

O Investimento 8 Será Utilizado

O Investimento 9 Não será Utilizado

O Investimento 10 Será Utilizado

O Investimento 11 Não será Utilizado

O Investimento 12 Não será Utilizado

O Investimento 13 Será Utilizado

O Investimento 14 Não será Utilizado

O Investimento 15 Será Utilizado

O Investimento 16 Não será Utilizado

O Investimento 17 Não será Utilizado

O Investimento 18 Não será Utilizado

O Investimento 19 Não será Utilizado

O Investimento 20 Não será Utilizado

Serão utilizados 4 investimentos



/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
import random
import numpy as np
import math

tamanho_pop = 1000
num_max_ger = 40
prevalencia = 180
tx_lider = 0.001
taxa_mutacao = 0.02
penal = 3

def sorteia_individuo(n):
  individuo = []
  for i in range(0,n):
    individuo.append(random.randint(0,1))
  return individuo

def fitness_func(ind):
  retorno = -fob(ind)
  if restricao_1(ind)>k:
    retorno = retorno/penal
  if restricao_2(ind)>raiz_capacidade:
    retorno = retorno/penal
  if restricao_3(ind)>b:
    retorno = retorno/penal
  return retorno

def reproducao(m):
  pro = []
  n = list(m)[:]
  q = list(m)[:]
  for i in range(1,math.ceil(tamanho_pop*tx_lider+1)):
    pro.append(list(n[-i]))

  while len(n)>1:
    aleat_1 = random.randint(0,len(n)-1)
    aleat_2 = random.randint(0,len(n)-1)
    pai = list(n[aleat_1])
    mae = list(n[aleat_2])
    fator_comb = random.randint(1,len(pai)-2)
    filho_1 = pai[0:fator_comb]+mae[fator_comb:]
    filho_2 = mae[0:fator_comb]+pai[fator_comb:]
    pro.append(filho_1)
    pro.append(filho_2)
    if pai!=mae:
      n.pop(aleat_1)
      if aleat_1>aleat_2:
        n.pop(aleat_2)
      else:
        n.pop(aleat_2-1)
    else:
      n.pop(aleat_2)
  for i in range(0,tamanho_pop-len(pro)):
    pai = list(q[random.randint(0,len(q)-1)])
    mae = list(q[random.randint(0,len(q)-1)])
    fator_comb = random.randint(1,len(pai)-2)
    filho = pai[0:fator_comb]+mae[fator_comb:]
    pro.append(filho)

  return pro

def mutacao(populacao, taxa_mutacao):
    nova_populacao = []
    for individuo in populacao:
        novo_individuo = individuo[:]
        if random.random() < taxa_mutacao:
          for i in range(len(novo_individuo)):
              if random.random() < taxa_mutacao:
                  novo_individuo[i] = 1 - novo_individuo[i]
        nova_populacao.append(novo_individuo)
    return nova_populacao

populacao = []
for i in range(0,tamanho_pop):
  populacao.append(sorteia_individuo(20))

cont_ger = 1

while cont_ger<=num_max_ger:
  retornos= []
  for i in range(0,tamanho_pop):
    retornos.append(fitness_func(populacao[i]))

  index_order = np.argsort(retornos)
  returns_in_order = np.array(retornos)[index_order]
  melhores = np.array(populacao)[index_order[-prevalencia:]]
  nv_pop = reproducao(melhores)
  nv_pop = mutacao(nv_pop,taxa_mutacao=taxa_mutacao)
  populacao = nv_pop

  cont_ger+=1
print(f'Valor Máximo = {max(retornos)}')
## tem hora que vai para 84 e tem hora que vai para 87
## a taxa de mutacao esta jogando os numero apenas para baixo pelo visto



Valor Máximo = 87
